In [ ]:
import numpy as np
import pandas as pd
from itertools import product
from tqdm import tqdm
import os
import sys

sys.path.append(os.path.abspath(os.path.join("..")))

import engine
import utils

pygame 2.5.2 (SDL 2.28.3, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [6]:
world_nums = sorted([int(file[6:-5]) for file in os.listdir("json")])
noise_range = np.arange(0.0, 1.1, 0.1)

In [14]:
full_grid = list(product(noise_range, noise_range, noise_range))
full_grid = [(np.round(x, 1), np.round(y, 1), np.round(z, 1)) for x, y, z in full_grid]
# remove parameters where there is no collision noise, but the col mean is not 1.0
full_grid = [parset for parset in full_grid if not (parset[2] == 0.0 and parset[1] != 1.0)]
len(full_grid)


1221

In [52]:
def run_sims(drop_noise, col_mean, col_sd, sims=100, world_nums=world_nums, print_prog=False):

    out_dict = {
        "world": [],
        "hole": [],
        "sim": [],
        "x_pos": []
    }

    for world_num in world_nums:
        if print_prog:
            print("World", world_num)
            
        for hole in [1,2,3]:
            world = utils.load_trial(world_num, 
                                     experiment="prediction", 
                                     hole=hole, 
                                     drop_noise=drop_noise, 
                                     col_mean=col_mean, 
                                     col_sd=col_sd)
            
            world["hole_dropped_into"] = hole-1
            
            for sim in range(sims):
                sim_data = engine.run_simulation(world, convert_coordinates=True)
                x_pos = sim_data["ball_position"][-1]["x"]

                out_dict["world"].append(world_num)
                out_dict["hole"].append(hole)
                out_dict["sim"].append(sim)
                out_dict["x_pos"].append(x_pos)

    return pd.DataFrame(out_dict)

In [ ]:
for parset in tqdm(full_grid):
    drop_noise, col_mean, col_sd = parset
    df = run_sims(drop_noise, col_mean, col_sd)
    df.to_csv(f"grid_search/prediction_{drop_noise}_drop_noise_{col_mean}_col_mean_{col_sd}_col_sd_100_sims.csv", index=False)